In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv")
faults.head()

C:\Users\nitin\AppData\Local\Temp\ipykernel_62612\3681478151.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [3]:
#removed rows from certina long and latititue . Lets go with 80 kms using haverson.
# Locations to drop around
drop_locations = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the distance in kilometers between two points on Earth 
    specified by their longitudes and latitudes in decimal degrees.
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 
    return c * r

# Mark rows to drop
radius_km = 30 #50 miles radius
def is_near_any(row, locations, radius):
    for lat, lon in locations:
        if haversine(row['Latitude'], row['Longitude'], lat, lon) <= radius:
            return True
    return False

# Apply and filter
faults['drop'] = faults.apply(lambda row: is_near_any(row, drop_locations, radius_km), axis=1)
faults_cleaned = faults[~faults['drop']].drop(columns='drop')

In [4]:
faults_selected = faults_cleaned[['RecordID','EventTimeStamp','eventDescription','spn','fmi','active','activeTransitionCount','EquipmentID','Latitude','Longitude','LocationTimeStamp']]

In [5]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [6]:
diag_pivoted = diagnostics.pivot(index='FaultId', columns='Name', values='Value').reset_index()

In [7]:
diag_pivoted

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [8]:
#Lets merge faults and diagnosis
faults_diag_merged = pd.merge(faults_selected, diag_pivoted, how='left', left_on='RecordID', right_on='FaultId')
faults_diag_merged

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917073,1248454,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,1761,17,False,3,2282,37.094768,-85.897407,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
917074,1248455,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.390740,-79.461805,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
917075,1248456,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.430370,-84.920509,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
917076,1248457,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,111,18,True,8,2377,35.030925,-85.321527,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [9]:
#convert time stamp from object to date
faults_diag_merged['EventTimeStamp'] = pd.to_datetime(faults_diag_merged['EventTimeStamp'],errors='coerce')
faults_diag_merged['LocationTimeStamp'] = pd.to_datetime(faults_diag_merged['LocationTimeStamp'],errors='coerce')
faults_targetted = faults_diag_merged[(faults_diag_merged['EventTimeStamp'] > '2019-01-01')]

In [10]:
#faults_targetted.sort_values(by='EventTimeStamp',ascending=True)
faults_targetted = faults_targetted[faults_targetted['spn'] != '5246']

### Grab the selected columns for target data sets

In [11]:
derate_codes = pd.read_csv("derate_codes.csv")

In [12]:
faults_target_data = faults_targetted[['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed',
'EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature',
'IgnStatus','IntakeManifoldTemperature','LampStatus','ServiceDistance','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure']]

In [13]:

(derate_codes['spn'].values)

array([   102,     91,    639,    111,    639,     94,     91,   1761,
         1761,   3362,   3251,   3216,   3226,   3482,    641,     96,
         3226,   3361,   3610,   3610,   3556,   3216,   4340,   4340,
         4342,   4765,   3242,   3246,     91,   4344,     84,    171,
         4094,   4096,   3361,   5394,   5394,   5394,   3362,   4334,
         4334,   4334,   4364,   3360,   3360,   5024,   3031,   1569,
         3216,   3216,   3226,   3364,   3364,   5848,   5742,   5743,
         5745,   1668,   3031,   1761,   6802,   1761, 520953,   4339,
          245,    251,   4094], dtype=int64)

### Here we will take each row from target data set where event date is < 12/30/2018 and based on the derate_codes, create new column ind_derate = 1 OR 0

In [14]:
def is_derate_codes(row, derate_codes):
    if((row['spn'] in list(derate_codes['spn'].values)) &
       (row['fmi'] in list(derate_codes[derate_codes['spn'] == row['spn']]['fmi'])) &
       (row['active'] == True)
      ):
       return 1
    else:
        return 0

In [15]:
#pass the codes
faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)

C:\Users\nitin\AppData\Local\Temp\ipykernel_62612\3759071785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)


In [16]:
faults_target_data['is_derate_code'].value_counts()

is_derate_code
0    80934
1    15711
Name: count, dtype: int64

### Now loop through the data and grab all diagnostic data for records where ind_derate = 1 per equipment ID. We will use these as predictors. 

In [17]:
faults_target_data

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
820219,1100564,1746,2019-01-01 00:24:10,111,18,True,0,14.355,0,73.4,...,60.8,True,59,17407,NaN,0,NaN,0,0.58,0
820220,1100565,1746,2019-01-01 00:36:08,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0
820221,1100566,1777,2019-01-01 01:54:35,111,18,True,0,13.9925,66.48672,113,...,NaN,True,87.8,18431,NaN,0,NaN,100,0,0
820222,1100567,1777,2019-01-01 02:08:21,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0
820223,1100568,1741,2019-01-01 03:06:52,2863,7,True,100,14.21,66.48672,183.2,...,NaN,True,105.8,255,NaN,66.89449,NaN,100,17.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917073,1248454,2282,2020-03-06 14:00:26,1761,17,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
917074,1248455,1994,2020-03-06 14:04:23,1569,31,True,100,14.5,64.6226,185,...,32,True,98.6,18431,NaN,65.01096,NaN,73.2,7.83,1
917075,1248456,1850,2020-03-06 14:13:38,3216,10,True,0,14.355,66.48672,186.8,...,NaN,True,91.4,17407,NaN,66.5741,NaN,100,6.96,1
917076,1248457,2377,2020-03-06 14:14:13,111,18,True,1.6,14.4275,67.72946,181.4,...,NaN,True,100.4,1023,NaN,11.84489,14.1,100,1.74,0


In [18]:
faults_target_data = faults_target_data.drop(['LampStatus','ServiceDistance'],axis = 1)

fault_test = faults_target_data.copy()

columns = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

for col in columns:

    fault_test[col] = pd.to_numeric(fault_test[col], errors='coerce')


    fault_test[col] = fault_test.groupby('EquipmentID')[col].transform(
        lambda x: x.fillna(x.mean())
    )


    fault_test[col] = fault_test[col].fillna(fault_test[col].mean())

In [19]:
fault_test = fault_test[fault_test['spn'] != '5246']
columns_to_use2 = ['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed','EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature','IgnStatus','IntakeManifoldTemperature','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure','is_derate_code']

try2 = fault_test[columns_to_use2]

columns = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

for col in columns:

    try2[col] = pd.to_numeric(try2[col], errors='coerce')


    try2[col] = try2.groupby('EquipmentID')[col].transform(
        lambda x: x.fillna(x.mean())
    )


    try2[col] = try2[col].fillna(try2[col].mean())

In [20]:
variables2 = ['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed','EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature', 'IntakeManifoldTemperature','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure','is_derate_code']
X_test = try2[variables2]
X_test

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,EngineTimeLtd,FuelLevel,FuelLtd,FuelTemperature,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
820219,1100564,1746,2019-01-01 00:24:10,111,18,True,0.000000,14.355000,0.000000,73.400000,...,11271.300000,64.000000,68243.830365,60.800000,59.000000,0.000000,13.594184,0.0,0.580000,0
820220,1100565,1746,2019-01-01 00:36:08,111,18,False,9.200000,14.065000,0.000000,123.800000,...,11331.975000,47.600000,68631.370765,81.500000,75.200000,29.386490,13.594184,9.2,0.870000,0
820221,1100566,1777,2019-01-01 01:54:35,111,18,True,0.000000,13.992500,66.486720,113.000000,...,10402.100000,87.600000,71141.401518,33.715815,87.800000,0.000000,13.594184,100.0,0.000000,0
820222,1100567,1777,2019-01-01 02:08:21,111,18,False,12.445714,14.373643,66.078390,149.051429,...,11437.327143,58.037500,78259.460850,33.715815,109.451429,1.996326,13.594184,100.0,1.002571,0
820223,1100568,1741,2019-01-01 03:06:52,2863,7,True,100.000000,14.210000,66.486720,183.200000,...,13285.100000,77.600000,70295.258435,33.715815,105.800000,66.894490,13.594184,100.0,17.110000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917073,1248454,2282,2020-03-06 14:00:26,1761,17,False,17.147826,14.427500,10.022987,167.313043,...,1136.177174,64.524444,7434.194930,33.715815,115.152174,1.949383,13.594184,100.0,1.248261,0
917074,1248455,1994,2020-03-06 14:04:23,1569,31,True,100.000000,14.500000,64.622600,185.000000,...,10722.700000,96.400000,58979.184416,32.000000,98.600000,65.010960,13.594184,73.2,7.830000,1
917075,1248456,1850,2020-03-06 14:13:38,3216,10,True,0.000000,14.355000,66.486720,186.800000,...,9326.750000,100.000000,65080.105870,32.000000,91.400000,66.574100,13.594184,100.0,6.960000,1
917076,1248457,2377,2020-03-06 14:14:13,111,18,True,1.600000,14.427500,67.729460,181.400000,...,586.750000,23.600000,4042.492826,33.715815,100.400000,11.844890,14.100000,100.0,1.740000,0


In [21]:
import joblib
pipeline = joblib.load('bigG_model_randomForest.pkl')
# 0.25 is threshold

#Predict with  pipeline and threshold
y_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred_final = (y_proba >= 0.25).astype(int)

# Map back to X_test
X_test_with_predictions = X_test.copy()
X_test_with_predictions['predicted_5246'] = y_pred_final
X_test_with_predictions['probability_5246'] = y_proba

# 3. See final result
X_test_with_predictions.head()

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLtd,FuelTemperature,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code,predicted_5246,probability_5246
820219,1100564,1746,2019-01-01 00:24:10,111,18,True,0.000000,14.355000,0.00000,73.400000,...,68243.830365,60.800000,59.000000,0.000000,13.594184,0.0,0.580000,0,0,0.00
820220,1100565,1746,2019-01-01 00:36:08,111,18,False,9.200000,14.065000,0.00000,123.800000,...,68631.370765,81.500000,75.200000,29.386490,13.594184,9.2,0.870000,0,0,0.02
820221,1100566,1777,2019-01-01 01:54:35,111,18,True,0.000000,13.992500,66.48672,113.000000,...,71141.401518,33.715815,87.800000,0.000000,13.594184,100.0,0.000000,0,0,0.09
820222,1100567,1777,2019-01-01 02:08:21,111,18,False,12.445714,14.373643,66.07839,149.051429,...,78259.460850,33.715815,109.451429,1.996326,13.594184,100.0,1.002571,0,0,0.16
820223,1100568,1741,2019-01-01 03:06:52,2863,7,True,100.000000,14.210000,66.48672,183.200000,...,70295.258435,33.715815,105.800000,66.894490,13.594184,100.0,17.110000,0,0,0.08


In [22]:
X_test_with_predictions[X_test_with_predictions['predicted_5246'] == 1]

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLtd,FuelTemperature,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code,predicted_5246,probability_5246
824680,1107742,1798,2019-01-16 11:44:02,1569,31,True,0.000000,14.355000,64.622600,186.800000,...,55792.212780,33.715815,87.800000,65.001250,13.594184,64.400000,1.740000,1,1,0.25
826442,1109902,2009,2019-01-21 13:47:31,102,18,False,31.428571,14.334286,66.220417,113.514286,...,36242.877110,33.715815,66.971429,15.101539,13.594184,98.457143,3.521429,0,1,0.27
833513,1119242,1758,2019-02-12 15:34:41,1569,31,True,33.600000,14.717500,62.137120,185.000000,...,65593.127995,33.715815,93.200000,65.379900,13.594184,24.400000,1.450000,1,1,0.25
834742,1121694,2009,2019-02-15 11:30:30,96,9,False,31.428571,14.334286,66.220417,113.514286,...,36242.877110,33.715815,66.971429,15.101539,13.594184,98.457143,3.521429,0,1,0.27
837316,1125048,1781,2019-02-22 11:37:54,5394,5,True,99.600000,14.500000,64.622600,188.600000,...,68995.664025,33.715815,82.400000,51.476720,13.594184,100.000000,11.600000,1,1,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889236,1208312,2026,2019-10-19 17:15:58,1761,19,True,31.200000,14.500000,46.602840,179.600000,...,50473.108513,33.715815,141.800000,4.563195,14.000000,100.000000,2.610000,1,1,0.27
895840,1217873,1812,2019-11-19 16:51:22,1569,31,True,75.200000,14.427500,64.622600,190.400000,...,61020.441861,33.715815,68.000000,63.020630,13.594184,100.000000,20.300000,1,1,0.25
897976,1220646,2003,2019-12-01 11:12:37,3362,31,True,58.800000,14.210000,66.486720,186.800000,...,52847.486917,33.715815,77.000000,63.175980,13.594184,100.000000,8.410000,1,1,0.29
912974,1241984,1814,2020-02-14 00:44:19,1569,31,True,72.000000,14.282500,64.622600,194.000000,...,73572.444826,33.715815,82.400000,63.117720,13.594184,30.000000,16.240000,1,1,0.28


In [23]:
faults_diag_merged[(faults_diag_merged['spn'] == 5246) &(faults_diag_merged['active'] == True) & (faults_diag_merged['EventTimeStamp'] > '2018-12-31')]  

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
823716,1105551,2019-01-13 21:21:37,NaN,5246,0,True,1,1698,34.698842,-81.938148,...,NaN,True,111.2,22527,True,NaN,0,NaN,100,0
826295,1109730,2019-01-21 09:01:38,NaN,5246,0,True,1,1827,39.950324,-83.339259,...,32,True,53.6,22527,NaN,NaN,0,NaN,100,0.29
827706,1111568,2019-01-25 10:06:17,NaN,5246,0,True,1,1857,34.118425,-84.741805,...,NaN,True,42.8,22527,True,NaN,0,NaN,100,0.58
828939,1113161,2019-01-29 22:35:39,NaN,5246,0,True,1,1758,30.478379,-90.460740,...,NaN,True,129.2,22527,True,NaN,0,NaN,100,0.29
829725,1114247,2019-01-31 21:23:21,NaN,5246,0,True,2,1758,35.137592,-90.487824,...,NaN,True,118.4,22527,True,NaN,0,NaN,100,0.29
829767,1114291,2019-02-01 02:42:48,NaN,5246,0,True,3,1758,33.829120,-93.396157,...,NaN,True,122,22527,True,NaN,0,NaN,100,0.29
830779,1115650,2019-02-05 03:22:15,NaN,5246,0,True,1,1918,32.601574,-95.921064,...,32,True,138.2,22527,True,NaN,0,NaN,100,0.29
831038,1116018,2019-02-05 19:59:02,NaN,5246,0,True,4,1758,39.725555,-86.220231,...,NaN,True,125.6,22527,True,NaN,0,NaN,100,0.29
831080,1116078,2019-02-06 02:31:49,NaN,5246,0,True,5,1758,38.491666,-85.769027,...,NaN,True,132.8,22527,True,NaN,0,NaN,100,0.29
836567,1124128,2019-02-20 18:24:28,NaN,5246,0,True,1,1811,36.797222,-87.392824,...,NaN,True,86,22527,NaN,NaN,0,NaN,100,0


In [25]:
from datetime import timedelta

X_test_with_predictions['real_5246'] = X_test_with_predictions.apply(
    lambda row: 1 if row['spn'] == 5246 and row['active'] == True else 0,
    axis=1
)


predictions_sort = X_test_with_predictions[['RecordID','EquipmentID','EventTimeStamp','spn','fmi','predicted_5246','real_5246']]
predictions_sort = predictions_sort[(predictions_sort['predicted_5246'] == 1) | (predictions_sort['real_5246'] == 1)]
unique_trucks = predictions_sort['EquipmentID'].unique()



def filter_within_hours(df, column, hours):
    df = df.sort_values('EventTimeStamp').copy()
    filtered = []
    next_allowed_time = None

    for idx, row in df.iterrows():
        current_time = row['EventTimeStamp']

        if row[column] == 1:
            if next_allowed_time is None or current_time >= next_allowed_time:
                filtered.append(1)
            else:
                filtered.append(0)
            
            next_allowed_time = current_time + timedelta(hours=hours)
        else:
            filtered.append(0)

    return filtered



def flag_missed_and_fake(df):
    df = df.sort_values('EventTimeStamp').reset_index(drop=True).copy()
    df['Missed'] = 0
    df['Fake'] = 0
    df['found'] = 0


    pred_indices = df[df['non_dupe_pred'] == 1].index
    real_indices = df[df['non_dupe_real'] == 1].index


    for idx in real_indices:
        event_time = df.loc[idx, 'EventTimeStamp']

        past_preds = df[(df['EventTimeStamp'] >= event_time - timedelta(hours=14)) & 
                        (df['EventTimeStamp'] < event_time) &
                        (df['non_dupe_pred'] == 1)]
        if past_preds.empty:
            df.at[idx, 'Missed'] = 1
        else:
            df.at[idx, 'found'] = 1

    for idx in pred_indices:
        event_time = df.loc[idx, 'EventTimeStamp']
        future_reals = df[(df['EventTimeStamp'] > event_time) & 
                          (df['EventTimeStamp'] <= event_time + timedelta(hours=14)) &
                          (df['non_dupe_real'] == 1)]
        if future_reals.empty:
            df.at[idx, 'Fake'] = 1

    return df




total_df_list = []

for i in unique_trucks:
    spec_truck = predictions_sort[predictions_sort['EquipmentID'] == i].copy()

    spec_truck['non_dupe_pred'] = filter_within_hours(spec_truck, 'predicted_5246', 12)
    spec_truck['non_dupe_real'] = filter_within_hours(spec_truck, 'real_5246', 12)
    
    spec_truck = flag_missed_and_fake(spec_truck)

    total_df_list.append(spec_truck)

total_df = pd.concat(total_df_list, ignore_index=True)

money_lost = (total_df['Fake'].sum() * 500)
# (total_df['Missed'].sum() * 4000) + 

money_saved = total_df['found'].sum() * 4000 

money_lost_og =  (total_df['Missed'].sum() * 4000) + (total_df['Fake'].sum() * 500)


print(f'money lost: ${money_lost}')
print(f'money saved: ${money_saved}')
print(f'Total Savings: ${money_saved-money_lost}')
print(f'money lost og: ${money_lost_og}')


print(f'found: {total_df['found'].sum()}')
print(f'Fake: {total_df['Fake'].sum()}')
print(f'Missed: {total_df['Missed'].sum()}')

money lost: $23500
money saved: $4000
Total Savings: $-19500
money lost og: $175500
found: 1
Fake: 47
Missed: 38
